In [44]:
import pandas as pd
import numpy as np
from datetime import datetime

In [71]:
from optionlab import Inputs
from optionlab import StrategyEngine
from optionlab.black_scholes import get_implied_vol, get_bs_info

Need custom implementaiton of BSM - theta values are incorrect

In [96]:
class CustomStrategy:
    def __init__(
            self,
            underlying:str,
            s0:float,
            r:float
        ):
        '''
        Custom options strategy containing n legs.
        Arguments:
            underlying (str): Underlying equity or index symbol.
            s0 (float): Current stock price.
            r (float): Current risk-free rate.

        '''
        self.underlying = underlying.upper()
        self.s0 = s0
        self.r = r
        self.legs = list()

    def add_leg(
        self,
        type:str,
        strike:float,
        contract_price:float,
        n_contracts:int,
        expiration:datetime,
    ):
        '''
        Define one leg of a strategy.
        Arguments:
            type (str): Contract type, either call or put
            strike (float): Strike price of the contract
            oprice (float): Current option contract price
            n_contracts (int): Number of contracts. Positive number denotes hold contract (buy), negative number denotes write contract (sell)
            expiration (datetime): Contract expiration date

        '''
        assert type in ['call', 'put'], "Contract type must be either 'call' or 'put'"

        dte = int(np.ceil(((expiration - datetime.today()).total_seconds() / (24*60*60))))
        assert dte > 0, "Contract must have positive time to expiration."

        leg = {
            'type': type,
            'strike': strike,
            'contract_price': contract_price,
            'n_contracts': n_contracts,
            'dte':dte,
        }
        
        leg['iv'] = self.compute_iv(leg)
        leg['greeks'] = self.compute_greeks(leg)

        self.legs.append(leg)

    def show_strategy(self):
        '''
        Prints out all the legs in the current strategy.
        '''
        for idx, leg in enumerate(self.legs):
            print(f"Leg {idx + 1}: {leg['position']} {leg['type']} with strike {leg['strike']} at price {leg['contract_price']}")

    def compute_iv(self, leg):
        return get_implied_vol(
            option_type=leg['type'], 
            oprice=leg['contract_price'], 
            s0 = self.s0, 
            x=leg['strike'], 
            r = self.r, 
            years_to_maturity=leg['dte']/365
            )
    
    def compute_greeks(self, leg):
        bs_out = get_bs_info(
            s=self.s0, 
            x=leg['strike'], 
            r=self.r, 
            vol=leg['iv'], 
            years_to_maturity=leg['dte']/365,
            y=0
        )
        if leg['type'] == 'call':
            return {
                'delta':bs_out.call_delta * leg['n_contracts'],
                'gamma':bs_out.gamma * leg['n_contracts'],
                'theta':bs_out.call_theta * leg['n_contracts'],
                'vega':bs_out.vega * leg['n_contracts'],
                }
        elif leg['type'] == 'put':
            return {
                'delta':bs_out.put_delta * leg['n_contracts'],
                'gamma':bs_out.gamma * leg['n_contracts'],
                'theta':bs_out.put_theta * leg['n_contracts'],
                'vega':bs_out.vega * leg['n_contracts'],
                }

In [111]:
strategy = CustomStrategy('pltr', 30.02, 0.0394)
strategy.add_leg('call', 31, 1.15, 1, datetime(2024, 11, 15))
strategy.legs

[{'type': 'call',
  'strike': 31,
  'option_price': 1.15,
  'n_contracts': 1,
  'dte': 96,
  'iv': np.float64(0.234),
  'greeks': {'delta': 0.4517166386965442,
   'gamma': 0.1099253758369025,
   'theta': -3.201338848315243,
   'vega': 0.06096963020923225}}]

In [75]:
s0 = 30.02 # stock price
x = 29.5 # strike
oprice = 1.15
r=0.04

In [ ]:
get_bs_info()

In [78]:
bs_out = get_bs_info(s0, x, r, 0.55, 6/365, )
bs_out.call_delta

0.6150004893443535

In [68]:
inputs_data = {
    "stock_price": 30.02,
    "start_date": "2024-08-11",
    "target_date": "2024-08-16",
    "volatility": 0.5576,
    "interest_rate": 0.0509,
    "min_stock": 25,
    "max_stock": 40,
    "strategy": [
        {
            "type": "call",
            "strike": 29,
            "premium": 1.52,
            "n": 1,
            "action":"buy"
        }
    ],
}

In [69]:
inputs = Inputs.model_validate(inputs_data,)
st = StrategyEngine(inputs,)
out = st.run()

ValidationError: 2 validation errors for Inputs
min_stock
  Field required [type=missing, input_value={'stock_price': 30.02, 's...': 1, 'action': 'buy'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
max_stock
  Field required [type=missing, input_value={'stock_price': 30.02, 's...': 1, 'action': 'buy'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

In [2]:
greeks = dict()
greeks['delta'] = 10.75
greeks['gamma'] = -1.57
greeks['theta'] = 0.083
greeks['vega'] = -0.41

In [19]:
lot_size = 1
multiplier = 100
iv = 0.3
s_0 = 37.58
dte = 33 # days
position_cost = -0.33
position_value = position_cost * lot_size * multiplier

In [50]:
s_min = round(s_0 * 0.99, 2)
s_max = round(s_0 * 1.01, 2)
price_range = np.arange(s_min, s_max, .05)
price_difference = price_range - s_0
adjusted_delta = greeks['delta'] + price_difference * greeks['gamma']
new_value = position_value + price_difference * adjusted_delta
pnl = new_value - position_value
delta_pnl = pd.DataFrame((price_range, pnl)).T
delta_pnl

,0,1
0,37.20,-4.311708
1,37.25,-3.718473
2,37.30,-3.133088
3,37.35,-2.555553
4,37.40,-1.985868
5,37.45,-1.424033
6,37.50,-0.870048
7,37.55,-0.323913
8,37.60,0.214372
9,37.65,0.744807
